# 서울시 버스 정류장 위치 표시

# 1 준비작업

## 1.1 Colab 과 GDrive 연동


In [0]:
use_colab = True

In [2]:
if use_colab == True:
  from google.colab import drive
  drive.mount("/content/gdrive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
if use_colab == True:
  work_root_path = "/content/gdrive/My Drive/JK/BusStation/"
else:
  work_root_path = "../"    # src 폴더 상위로 올라감
  
input_path = work_root_path+"dataset/"
output_path = work_root_path+"output/"

In [4]:
cd '/content/gdrive/My Drive'

/content/gdrive/My Drive


## 1.2 Import

In [0]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster, MiniMap

# 2 지도 표시

## 2.1 서울시청 위도, 경도 좌표 파악
구글 맵스 등을 이용하여 서울시청의 위도, 경도를 파악한다.

위도(latitude) : 37.565954

경도(longitude) : 126.978023

## 2.2 서울시 지도 표시

In [6]:
# 서울시청 위도, 경도
latitude_value = 37.565954
longitude_value = 126.978023

map_hs = folium.Map((latitude_value, longitude_value), zoom_start = 11)
mc = MarkerCluster()

mc.add_child(folium.Marker(location = [latitude_value, longitude_value],
                           popup = '서울시청',
                           icon = folium.Icon(color = 'red', icon = 'info-sign')))

minimap = MiniMap()
map_hs.add_child(minimap)

map_hs.add_child(mc)

## 2.3 버스 정류장 데이타 가져오기

공공 데이타 포털에서 서울시 버스 정류소 좌표 데이타를 가져와서 seoul_bus_stations.csv 파일을 만든다.

http://data.seoul.go.kr/dataList/OA-15067/S/1/datasetView.do;jsessionid=799032EC8DB367BD3563D6CC38A2DFA4.new_portal-svr-21

이때, 컬럼명은 프로그래밍상의 편의를 위해서 ars_id, standard_id, station_name, x_coord, y_coord 로 변경한다.

In [7]:
input_file = input_path+'seoul_bus_stations.csv'
print(input_file)

# utf-8 오류 발생하여 CP949 로 교체함
#bus_stations = pd.read_csv(input_file, encoding = 'utf-8')
bus_stations = pd.read_csv(input_file, encoding ='CP949')

/content/gdrive/My Drive/JK/BusStation/dataset/seoul_bus_stations.csv


In [8]:
bus_stations.head()

,ars_id,standard_id,station_name,x_coord,y_coord
0,1001,100000001,종로2가사거리,126.987786,37.569764
1,1002,100000002,창경궁.서울대학교병원,126.996520,37.579179
2,1003,100000003,명륜3가.성대입구,126.998290,37.582709
3,1004,100000004,종로2가.삼일교,126.987507,37.568582
4,1005,100000005,혜화동로터리.여운형활동터,127.001694,37.586230


## 2.4 지도 표시용 데이타 변환

지도에 표시하기 위해서 필요한 위도, 경도, 정류소 이름을 추출한다.

In [0]:
loc_data = bus_stations[['y_coord','x_coord','station_name']].drop_duplicates(keep = 'first')
loc_data = bus_stations.groupby(['station_name'])['y_coord','x_coord'].mean()

In [0]:
loc_data = loc_data.reset_index()

In [11]:
loc_data.head()

,station_name,y_coord,x_coord
0,(구)동양마트앞,37.541551,127.090017
1,123전자타운.2001아울렛,37.497222,126.862107
2,12번지건영아파트,37.629075,126.914023
3,14단지상가,37.669808,127.059583
4,1호선구일역,37.495350,126.872224


In [12]:
loc_data.shape

(6884, 3)

## 2.5 지도에 정류소 표시

itertuples 를 사용하여 모든 정류소를 표시한다.

In [13]:
# 서울시청 위도, 경도
latitude_value = 37.565954
longitude_value = 126.978023

map_hs = folium.Map((latitude_value, longitude_value), zoom_start = 11)
mc = MarkerCluster()

idx = 0
for row in loc_data.itertuples():
    mc.add_child(folium.Marker(location = [row.y_coord, row.x_coord], 
                               popup = row.station_name))

minimap = MiniMap()
map_hs.add_child(minimap)

map_hs.add_child(mc)

Output hidden; open in https://colab.research.google.com to view.

정류장이 1개이면 파란색으로 표시되고, 여러개가 모여 있으면 숫자로 표시된다.

## 2.6 지표 표시 내용을 html 로 저장

Folium 을 사용할 때 하얗게 출력되면 표시할 것이 많은 경우이므로 html로 저장해서 확인하면 된다.

In [0]:
map_hs.save(output_path+'bus_stations.html')